In [1]:
import embedding_functions
import functions
from PyPDF2 import PdfFileReader
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import pandas as pd
import openai
openai.api_key_path = "API.csv"

/home/gsilvi/miniforge3/envs/PaperGenie/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:


#complete_text = functions.extract_all_text(['papers/1801.00862/NISQ-Quantum.tex']) 
complete_text = functions.extract_all_text(['papers/2201.08194/BP_shadows.tex']) 
#complete_text = functions.extract_all_text(['papers/2002.12902/main.tex']) 


In [3]:
final_text = embedding_functions.texStripper(complete_text)

dict_keys(['pre_section', 'Introduction', 'Avoiding barren plateaus in variational quantum optimization', 'Variational quantum eigensolver', 'Barren plateaus and entanglement', 'Weak barren plateaus and improved algorithm', 'Weak barren plateaus and initialization of VQE', 'Definition and relation to barren plateaus ', 'Illustration of WBP-free initialization', 'Entanglement control during optimization', 'Bounding entanglement increase at a single optimization step', 'Optimization performance with learning rate', 'Summary and Discussion', 'Acknowledgments', 'Appendix', 'Classical shadows and implementation details', 'Data acquisition via classical shadows', 'Estimating subsystem purities', 'Estimating gradients', 'Example of error accumulation in an Ising model', 'Proof of Theorem~\\ref{thm:linear-shadows} \\labelSec:proof', 'Unitary \\texorpdfstring{$t$}--designs}\\labelappx:t-design', 'Entanglement and unitary \\texorpdfstring{$2$}--designs}\\labelapp:ent', 'Bounding the expected tra

In [4]:
final_text['full']

['title: Avoiding barren plateaus using classical shadows',
 'author: Stefan H. Sack,Raimel A. Medina,Alexios A. Michailidis,Richard Kueng,Maksym Serbyn',
 'thanks: Equal contribution,Equal contribution',
 'email: stefan.sack@ist.ac.at,raimel.medina@ist.ac.at',
 'affiliation: IST Austria, Am Campus 1, 3400 Klosterneuburg, Austria,IST Austria, Am Campus 1, 3400 Klosterneuburg, Austria,Department of Theoretical Physics, University of Geneva,24 quai Ernest-Ansermet, 1211 Geneva, Switzerland,Institute for Integrated Circuits, Johannes Kepler University Linz, Austria,IST Austria, Am Campus 1, 3400 Klosterneuburg, Austria',
 'Abstract: Variational quantum algorithms are promising algorithms for achieving quantum advantage on near-term devices. The quantum hardware is used to implement a variational wave function and measure observables, whereas the classical computer is used to store and update the variational parameters. The optimization landscape of expressive variational ans\\"atze is how

In [5]:
df = pd.DataFrame(final_text['full'], columns=['Phrase'])
df

,Phrase
0,title: Avoiding barren plateaus using classica...
1,"author: Stefan H. Sack,Raimel A. Medina,Alexio..."
2,"thanks: Equal contribution,Equal contribution"
3,"email: stefan.sack@ist.ac.at,raimel.medina@ist..."
4,"affiliation: IST Austria, Am Campus 1, 3400 Kl..."
...,...
401,This conjecture was subsequently proven in Ref...
402,"Putting everything together, we obtain\be-\ln ..."
403,Here we detail the derivation of Eq.~(\ref{eq:...
404,We first upper bound the trace distance via\be...


In [6]:
embedding_functions.save_embedding(df,'2201.08194')


Token indices sequence length is longer than the specified maximum sequence length for this model (1328 > 1024). Running this sequence through the model will result in indexing errors


Max tokens 1672


Read file from here

In [2]:
from operator import index
import pandas as pd
import numpy as np

df = pd.read_csv('embeddings/text-search-babbage-doc-001_2201.08194.csv', index_col=0)
df['similarity'] = df.similarity.apply(eval).apply(np.array)
df['search'] = df.search.apply(eval).apply(np.array)




In [4]:
import embedding_functions
question = "Conclusions "
df_sorted,newres = embedding_functions.search_phrases(df, question, how_many_std=3,connect_adj=True)

Mean 0.24322938048661125 1 std 0.25860110365730254 2 std 0.2739728268279938 Filter similarity: 0.2893445499986851
This conjecture was subsequently proven in Ref.~\cite{proofPage_1994} 0.2928347836741071



In [10]:
import difflib


complete_text = functions.extract_all_text(['papers/2201.08194/BP_shadows.tex']) 
for phrase in newres.Phrase:
    print(phrase)
    print(phrase in complete_text)
    print(difflib.get_close_matches(phrase, complete_text,n=1, cutoff=0.1))
    

The algorithm employs  \emph{classical shadow} estimation~\cite{huang2020predicting} during the optimization process in order to efficiently estimate the expectation value of the cost function, its  gradients, and the second R\'enyi entropy of small subsystems. The tracking of the second R\'enyi entropy enabled by the classical shadows protocol allows for an efficient diagnosis of the WBP both at the initialization step  and during the optimization process of variational parameters
False
[]
Using classical shadows we can efficiently check all subregions of size $k$ with only a $k \ln(N)$-overhead in the measurement budget, see Appendix~\ref{appx:shadows} for details
True
[]
\citeauthor{huang2020predicting} developed a more near-term friendly variant of this general idea known as prediction with \emph{classical shadows}
True
[]
   \item (Implicitly) construct the $N$-qubit \emph{classical shadow}    \begin{equation}    \hat{\rho}^{(t)} (\bm{\theta}) =      \bigotimes_{i=1}^N \left( 3 |s

In [22]:
if True:
    import fitz
    pdfile = 'papers/2201.08194/BP_shadows.pdf'
    ### READ IN PDF
    doc = fitz.open(pdfile)

    phrase_to_search = 'Variational quantum algorithms are promising algorithms for achieving \quantum advantage on near-term devices.'

    def highlight_pdf(doc, phrase_to_search):
        middle =len(phrase_to_search)//2
        phrase_split1 = phrase_to_search.split(' ')
        phrase_split2 = phrase_split1.copy()
        phrase_found_1 = False
        phrase_found_2 = False
        phrase2=None
        while phrase_found_1 == False or phrase_found_2 == False: 
            phrase1 = ' '.join(phrase_split1)
            phrase2 = ' '.join(phrase_split2)
            for page in doc:
                ### SEARCH
                if phrase_found_1 == False:
                    text_instances =page.search_for(phrase1)
                    if text_instances != []:
                        print(text_instances)
                        highlight = page.add_highlight_annot(text_instances)
                        highlight.update()
                        phrase_found_1 = True
                if phrase_found_2 == False:
                    text_instances =page.search_for(phrase2)
                    if text_instances != []:
                        print(text_instances)
                        highlight = page.add_highlight_annot(text_instances)
                        highlight.update()
                        phrase_found_2 = True
                
            if phrase_found_1 == False:
                phrase_split1 = phrase_split1[:-1]
            if phrase_found_2 == False:
                phrase_split2 = phrase_split2[1:]
        
            

    for phrase_to_search in newres.Phrase:  
        highlight_pdf(doc, phrase_to_search)
            

    ### OUTPUT
    doc.save("output.pdf", garbage=4, deflate=True, clean=True)

[Rect(142.029541015625, 461.955078125, 279.1746520996094, 471.9176940917969)]
[Rect(176.78903198242188, 427.6661071777344, 219.81748962402344, 437.62872314453125)]
[Rect(175.09535217285156, 415.607421875, 299.099853515625, 425.5700378417969), Rect(53.99996566772461, 427.0644226074219, 299.0500183105469, 437.02703857421875)]
[Rect(81.68602752685547, 438.52142333984375, 299.1098327636719, 448.4840393066406), Rect(53.99996566772461, 449.9784240722656, 113.86521911621094, 459.9410400390625)]
None
None


KeyboardInterrupt: 

[Rect(118.00994873046875, 151.95025634765625, 311.6035461425781, 160.91665649414062)]

# Try directly from pdf

In [6]:
#open a local pdf file and convert it with PDfFileReader
pdf = PdfFileReader(open( 'papers/2201.08194/BP_shadows.pdf', "rb")) # open the pdf file, rb is for read binary, other options are r, w, a that mean read, write, append

text = ""
for page in range(pdf.getNumPages()-3):
    text += pdf.getPage(page).extractText()
    print(page)
# text.replace('-\n', '').replace('\n', ' ').split('.')
#keep only the elements longer than 5 characters
phrases = [section for section in text.replace('-\n', '').replace('i.e.', 'id est').replace('\n', ' ').split('. ') if len(section) > 6]
len(phrases)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


503

In [9]:
phrases[2]

'Medina,1,yAlexios A'